<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVISProject-CAValli_Team/blob/main/Preprocessing_story2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading the data

In [2]:
all_files = glob.glob("/content/drive/MyDrive/DV_project/story2/*.xlsx")
all_files.sort()
print(all_files)

['/content/drive/MyDrive/DV_project/story2/early_leavers_by_sex_labour.xlsx', '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_absolute.xlsx', '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_percentage.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk_by_age.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk_by_education.xlsx']


## Education level: absolute value

We import the 36 sheets of the *pop_edu_sex_age_absolute* excel file (skipping useless rows/columns and avoiding the print of unharmful warnings) in a dictionary of datasets named *education_abs_sheets*.

In [3]:
sheet_names = [f'Sheet {i}' for i in range(1, 37)]
file_path = '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_absolute.xlsx'
rows_to_skip = list(range(0, 10)) + [11, 12, 13, 44] + list(range(49, 58))
selected_columns = ['TIME'] + [str(year) for year in range(2009, 2023)]

# Just to suppress the unharmful warnings we were getting
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = UserWarning, module = "openpyxl")
    # Dictionary with datasets: 'education_abs_sheet1', ..., 'education_abs_sheet36'
    education_abs_sheets = {f'education_abs_sheet{i}': pd.read_excel(file_path, sheet_name = sheet_name, skiprows = rows_to_skip, na_values = ':', usecols = selected_columns)
                            for i, sheet_name in enumerate(sheet_names, start = 1)}

We create 2 dictionaries linking each country to its short abbreviation (*country_abbreviations*) and viceversa (*country_abbreviations_reversed*).

In [4]:
country_abbreviations = {
    'Belgium': 'BEL',
    'Bulgaria': 'BGR',
    'Czechia': 'CZE',
    'Denmark': 'DNK',
    'Germany': 'DEU',
    'Estonia': 'EST',
    'Ireland': 'IRL',
    'Greece': 'GRC',
    'Spain': 'ESP',
    'France': 'FRA',
    'Croatia': 'HRV',
    'Italy': 'ITA',
    'Cyprus': 'CYP',
    'Latvia': 'LVA',
    'Lithuania': 'LTU',
    'Luxembourg': 'LUX',
    'Hungary': 'HUN',
    'Malta': 'MLT',
    'Netherlands': 'NLD',
    'Austria': 'AUT',
    'Poland': 'POL',
    'Portugal': 'PRT',
    'Romania': 'ROU',
    'Slovenia': 'SVN',
    'Slovakia': 'SVK',
    'Finland': 'FIN',
    'Sweden': 'SWE',
    'Iceland': 'ISL',
    'Norway': 'NOR',
    'Switzerland': 'CHE',
    'Montenegro': 'MNE',
    'North Macedonia': 'MKD',
    'Serbia': 'SRB',
    'Türkiye': 'TUR',
}

country_abbreviations_reversed = {value: key for key, value in country_abbreviations.items()}

We add a column *Abbreviation* to the 36 datasets.

In [5]:
for i in range(1, 37):
    education_abs_sheets[f'education_abs_sheet{i}'].rename(columns = {'TIME': 'Country'}, inplace = True)
    education_abs_sheets[f'education_abs_sheet{i}']['Abbreviation'] = education_abs_sheets[f'education_abs_sheet{i}']['Country'].map(country_abbreviations)

We sum data about 15-24 y.o. people and data about 25-29 y.o. people to have data coherent to what we used in story 1 (people aged between 15 and 29).

We create 3 different dataset, one for each educational level (low, medium and high).

In [42]:
low1 = education_abs_sheets['education_abs_sheet2']
low2 = education_abs_sheets['education_abs_sheet8']

medium1 = education_abs_sheets['education_abs_sheet3']
medium2 = education_abs_sheets['education_abs_sheet9']

high1 = education_abs_sheets['education_abs_sheet6']
high2 = education_abs_sheets['education_abs_sheet12']

In [43]:
low1.set_index(['Country', 'Abbreviation'], inplace = True)
low2.set_index(['Country', 'Abbreviation'], inplace = True)
low = low1.fillna(1000000).add(low2.fillna(1000000), fill_value = 0).round(1)
low = low.where(low < 1000000)
low1.reset_index(inplace = True)
low2.reset_index(inplace = True)
low.reset_index(inplace = True)

In [44]:
medium1.set_index(['Country', 'Abbreviation'], inplace = True)
medium2.set_index(['Country', 'Abbreviation'], inplace = True)
medium = medium1.fillna(1000000).add(medium2.fillna(1000000), fill_value = 0).round(1)
medium = medium.where(medium < 1000000)
medium1.reset_index(inplace = True)
medium2.reset_index(inplace = True)
medium.reset_index(inplace = True)

In [45]:
high1.set_index(['Country', 'Abbreviation'], inplace = True)
high2.set_index(['Country', 'Abbreviation'], inplace = True)
high = high1.fillna(1000000).add(high2.fillna(1000000), fill_value = 0).round(1)
high = high.where(high < 1000000)
high1.reset_index(inplace = True)
high2.reset_index(inplace = True)
high.reset_index(inplace = True)

We create one dataset for each year to use for the stacked barchart (percentage).

In [51]:
def create_stacked_dataset(year):
    stacked = pd.DataFrame()

    low_year = low[['Country', 'Abbreviation', year]].rename(columns = {year: 'low'})
    medium_year = medium[['Country', 'Abbreviation', year]].rename(columns = {year: 'medium'})
    high_year = high[['Country', 'Abbreviation', year]].rename(columns = {year: 'high'})

    stacked = pd.merge(low_year, medium_year, on = ['Country', 'Abbreviation'])
    stacked = pd.merge(stacked, high_year, on = ['Country', 'Abbreviation'])
    stacked = stacked.dropna(subset = ['low', 'medium', 'high'])

    return stacked

for year in range(2009, 2023):
    stacked_year = create_stacked_dataset(str(year))
    stacked_year = stacked_year.sort_values(by = 'Country')
    stacked_year.to_csv(f"stacked{year}.csv", index = False, na_rep = 'nan')

for year in range(2009, 2023):
    stacked_year = create_stacked_dataset(str(year))
    stacked_year = stacked_year.sort_values(by = 'low', ascending = False)
    stacked_year.to_csv(f"stacked{year}_low.csv", index = False, na_rep = 'nan')

We do the same for males.

In [26]:
low1_M = education_abs_sheets['education_abs_sheet14']
low2_M = education_abs_sheets['education_abs_sheet20']

medium1_M = education_abs_sheets['education_abs_sheet15']
medium2_M = education_abs_sheets['education_abs_sheet21']

high1_M = education_abs_sheets['education_abs_sheet18']
high2_M = education_abs_sheets['education_abs_sheet24']

In [28]:
low1_M.set_index(['Country', 'Abbreviation'], inplace = True)
low2_M.set_index(['Country', 'Abbreviation'], inplace = True)
low_M = low1_M.fillna(1000000).add(low2_M.fillna(1000000), fill_value = 0).round(1)
low_M = low_M.where(low_M < 1000000)
low1_M.reset_index(inplace = True)
low2_M.reset_index(inplace = True)
low_M.reset_index(inplace = True)
low_M

,Country,Abbreviation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,BEL,383.2,391.7,395.0,393.1,387.9,376.6,376.6,370.3,379.1,387.0,386.1,384.3,348.5,334.0
1,Bulgaria,BGR,289.8,249.6,228.6,214.8,206.9,205.7,207.2,201.9,192.3,191.8,191.0,191.3,181.0,165.0
2,Czechia,CZE,335.8,316.3,303.6,288.4,276.7,265.4,263.5,266.3,263.2,265.6,275.1,279.9,280.0,287.0
3,Denmark,DNK,262.2,275.1,271.0,268.3,270.4,267.5,267.6,255.2,262.7,265.1,258.0,258.2,253.9,266.0
4,Germany,DEU,3164.3,2933.0,2079.8,2037.8,2005.6,2761.8,2799.7,2823.5,2827.6,2763.2,2723.4,2663.0,2762.9,2995.5
5,Estonia,EST,58.4,55.5,51.2,49.3,46.4,47.1,45.2,45.2,44.4,42.6,40.9,40.2,39.6,40.7
6,Ireland,IRL,162.4,161.2,157.8,149.6,147.2,138.8,138.6,139.2,132.0,134.0,137.9,136.9,135.4,138.3
7,Greece,GRC,411.7,390.6,383.6,353.1,325.0,301.2,276.6,258.6,257.0,247.4,242.4,236.5,247.5,239.4
8,Spain,ESP,2162.3,2054.5,1975.4,1888.6,1800.0,1705.6,1640.6,1570.2,1533.2,1503.7,1491.6,1452.7,1638.7,1648.7
9,France,FRA,2014.6,2060.2,2031.6,2010.5,1880.9,1877.7,1883.0,1896.4,1885.5,1880.5,1860.5,1800.6,1871.2,1874.5


In [29]:
medium1_M.set_index(['Country', 'Abbreviation'], inplace = True)
medium2_M.set_index(['Country', 'Abbreviation'], inplace = True)
medium_M = medium1_M.fillna(1000000).add(medium2_M.fillna(1000000), fill_value = 0).round(1)
medium_M = medium_M.where(medium_M < 1000000)
medium1_M.reset_index(inplace = True)
medium2_M.reset_index(inplace = True)
medium_M.reset_index(inplace = True)
medium_M

,Country,Abbreviation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,BEL,435.7,429.1,435.1,433.0,435.4,442.8,445.4,456.7,422.7,418.4,417.4,412.0,436.7,469.5
1,Bulgaria,BGR,380.7,412.8,409.0,388.5,370.8,347.5,329.5,315.1,297.5,282.8,272.0,254.2,257.9,270.4
2,Czechia,CZE,652.2,637.1,598.9,578.6,559.3,542.1,527.7,495.4,479.1,470.3,463.7,453.4,431.5,422.2
3,Denmark,DNK,173.5,172.3,177.7,184.7,188.4,197.2,204.0,210.6,202.8,211.2,213.5,215.2,210.4,211.2
4,Germany,DEU,3476.6,3341.8,3343.0,3357.0,3360.8,3413.7,3365.9,3505.9,3464.4,3368.1,3265.0,3181.5,2971.4,2810.4
5,Estonia,EST,67.6,67.4,66.2,63.0,63.5,57.5,56.7,56.6,54.1,52.1,51.4,47.8,44.9,45.5
6,Ireland,IRL,222.4,210.9,200.4,193.9,185.1,186.6,183.8,187.7,193.1,203.1,203.3,207.5,206.4,210.4
7,Greece,GRC,485.8,471.2,445.8,439.6,430.0,436.5,456.6,471.2,447.8,455.0,461.0,464.8,458.0,445.9
8,Spain,ESP,1132.5,1120.9,1082.0,1035.3,1006.4,1031.9,1038.7,1045.1,1064.3,1091.3,1137.8,1204.6,1245.2,1276.0
9,France,FRA,2404.8,2366.3,2356.5,2322.2,2416.3,2507.6,2510.6,2502.9,2525.5,2453.7,2434.2,2475.2,2294.5,2412.9


In [30]:
high1_M.set_index(['Country', 'Abbreviation'], inplace = True)
high2_M.set_index(['Country', 'Abbreviation'], inplace = True)
high_M = high1_M.fillna(1000000).add(high2_M.fillna(1000000), fill_value = 0).round(1)
high_M = high_M.where(high_M < 1000000)
high1_M.reset_index(inplace = True)
high2_M.reset_index(inplace = True)
high_M.reset_index(inplace = True)
high_M

,Country,Abbreviation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,BEL,188.8,190.3,184.6,191.5,197.2,202.1,204.6,200.2,219.7,220.9,223.4,231.8,244.5,234.3
1,Bulgaria,BGR,48.8,56.5,57.8,66.5,70.9,75.4,74.0,73.0,78.3,72.0,63.5,62.4,59.9,55.8
2,Czechia,CZE,98.5,103.2,102.8,112.6,115.7,117.3,109.8,117.3,117.3,108.7,94.5,90.2,101.4,91.1
3,Denmark,DNK,45.4,44.3,48.5,52.9,55.7,62.9,69.3,78.3,81.4,80.0,87.7,87.0,89.6,87.0
4,Germany,DEU,561.5,554.9,607.8,650.2,683.2,706.6,765.2,865.2,895.2,884.8,916.6,953.8,958.8,992.8
5,Estonia,EST,16.3,16.3,18.4,18.1,17.5,18.2,16.2,14.9,14.7,15.2,14.3,15.3,16.0,13.8
6,Ireland,IRL,113.5,104.3,95.5,95.8,97.4,97.7,101.6,100.3,107.3,106.4,109.7,113.3,120.8,125.7
7,Greece,GRC,120.3,113.6,122.3,132.9,135.0,130.2,125.6,119.4,114.7,109.3,114.1,120.3,138.2,148.5
8,Spain,ESP,793.9,749.5,717.7,709.5,690.1,655.9,657.1,682.8,701.4,744.6,778.1,811.9,867.1,893.2
9,France,FRA,1192.8,1160.8,1164.8,1181.3,1193.4,1259.9,1242.9,1244.3,1231.3,1301.5,1333.7,1372.6,1428.8,1431.6


In [15]:
low_M.to_csv('low_M.csv', index = False, na_rep = 'nan')
medium_M.to_csv('medium_M.csv', index = False, na_rep = 'nan')
high_M.to_csv('high_M.csv', index = False, na_rep = 'nan')

We do the same for females.

In [31]:
low1_F = education_abs_sheets['education_abs_sheet26']
low2_F = education_abs_sheets['education_abs_sheet32']

medium1_F = education_abs_sheets['education_abs_sheet27']
medium2_F = education_abs_sheets['education_abs_sheet33']

high1_F = education_abs_sheets['education_abs_sheet30']
high2_F = education_abs_sheets['education_abs_sheet36']

In [32]:
low1_F.set_index(['Country', 'Abbreviation'], inplace = True)
low2_F.set_index(['Country', 'Abbreviation'], inplace = True)
low_F = low1_F.fillna(1000000).add(low2_F.fillna(1000000), fill_value = 0).round(1)
low_F = low_F.where(low_F < 1000000)
low1_F.reset_index(inplace = True)
low2_F.reset_index(inplace = True)
low_F.reset_index(inplace = True)
low_F

,Country,Abbreviation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,BEL,328.6,342.9,334.1,327.4,325.7,317.3,321.2,320.1,329.0,334.1,329.3,326.3,288.5,291.0
1,Bulgaria,BGR,284.0,245.3,225.7,211.2,207.0,199.7,197.0,194.6,190.6,175.3,181.0,183.9,172.4,161.5
2,Czechia,CZE,315.3,304.9,289.9,270.6,254.5,243.6,247.1,255.3,250.9,252.2,252.9,264.7,259.7,261.3
3,Denmark,DNK,211.2,222.6,221.5,217.6,217.8,221.3,218.6,219.2,210.4,214.3,217.3,213.4,213.0,217.4
4,Germany,DEU,2859.2,2652.2,1884.1,1848.2,1774.5,2439.0,2452.0,2402.9,2384.7,2348.7,2346.2,2185.0,2351.5,2576.0
5,Estonia,EST,46.5,42.5,41.2,39.0,35.5,33.2,36.6,34.2,33.5,33.8,34.4,33.7,33.9,35.1
6,Ireland,IRL,134.3,140.4,134.4,131.4,129.8,123.2,126.2,120.0,119.2,127.4,126.6,129.3,119.7,126.6
7,Greece,GRC,322.7,313.8,306.9,287.8,274.2,249.6,254.0,242.1,239.4,236.4,228.9,231.4,216.9,224.7
8,Spain,ESP,1601.5,1523.0,1463.6,1422.5,1383.8,1333.2,1237.2,1203.9,1195.7,1153.7,1103.5,1083.6,1285.1,1304.6
9,France,FRA,1815.3,1794.6,1805.9,1753.3,1683.4,1691.2,1726.3,1683.1,1706.8,1690.8,1657.4,1596.2,1641.7,1672.6


In [33]:
medium1_F.set_index(['Country', 'Abbreviation'], inplace = True)
medium2_F.set_index(['Country', 'Abbreviation'], inplace = True)
medium_F = medium1_F.fillna(1000000).add(medium2_F.fillna(1000000), fill_value = 0).round(1)
medium_F = medium_F.where(medium_F < 1000000)
medium1_F.reset_index(inplace = True)
medium2_F.reset_index(inplace = True)
medium_F.reset_index(inplace = True)
medium_F

,Country,Abbreviation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,BEL,400.0,390.9,402.5,400.9,399.1,394.3,398.8,398.1,375.1,356.0,363.4,351.8,375.1,399.1
1,Bulgaria,BGR,295.9,326.4,326.3,319.8,297.4,279.0,269.2,253.9,239.2,238.5,228.8,218.3,219.2,222.3
2,Czechia,CZE,571.3,547.0,519.5,490.5,470.6,464.5,433.7,409.5,394.4,380.1,382.9,357.7,356.1,345.0
3,Denmark,DNK,178.9,173.4,177.1,182.4,181.4,188.1,193.8,198.5,200.1,200.8,197.0,201.8,199.9,196.1
4,Germany,DEU,3345.7,3273.5,3186.7,3105.6,3126.5,3284.3,3188.5,3226.5,3168.9,3107.9,3031.7,3022.5,2754.3,2594.7
5,Estonia,EST,58.3,58.8,58.4,54.7,54.6,54.7,50.6,48.1,47.0,46.3,42.7,42.8,39.9,38.4
6,Ireland,IRL,206.1,187.3,185.6,179.6,166.8,166.9,158.9,163.8,175.9,174.7,186.1,180.3,180.2,185.4
7,Greece,GRC,479.8,461.1,427.1,416.6,412.6,423.1,403.3,399.7,387.5,386.5,385.9,374.5,384.0,384.6
8,Spain,ESP,1240.6,1186.5,1150.3,1129.2,1086.1,1069.2,1119.5,1109.7,1088.5,1130.4,1189.0,1212.7,1238.3,1230.0
9,France,FRA,2307.2,2339.2,2331.4,2340.3,2336.6,2424.8,2431.3,2432.5,2407.3,2365.8,2350.1,2360.9,2230.2,2253.7


In [34]:
high1_F.set_index(['Country', 'Abbreviation'], inplace = True)
high2_F.set_index(['Country', 'Abbreviation'], inplace = True)
high_F = high1_F.fillna(1000000).add(high2_F.fillna(1000000), fill_value = 0).round(1)
high_F = high_F.where(high_F < 1000000)
high1_F.reset_index(inplace = True)
high2_F.reset_index(inplace = True)
high_F.reset_index(inplace = True)
high_F

,Country,Abbreviation,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Belgium,BEL,263.0,261.8,263.5,275.0,280.8,294.5,287.7,290.0,299.6,312.7,310.1,324.1,338.8,322.1
1,Bulgaria,BGR,87.5,104.7,102.3,101.0,108.7,115.1,111.7,110.4,108.6,104.3,90.4,80.5,82.6,82.2
2,Czechia,CZE,131.8,140.8,144.0,169.3,180.2,173.0,178.5,173.5,174.6,170.8,152.8,154.5,150.4,145.5
3,Denmark,DNK,71.8,73.8,78.5,81.5,87.0,96.2,104.1,102.9,113.2,118.4,121.3,123.6,124.4,130.0
4,Germany,DEU,754.4,741.8,842.8,910.4,921.3,823.8,883.0,942.4,969.0,973.8,994.1,1107.6,1137.5,1166.1
5,Estonia,EST,32.0,32.5,30.5,32.7,32.4,30.4,26.8,30.0,28.3,25.7,26.0,23.5,23.8,23.4
6,Ireland,IRL,164.6,153.4,141.6,137.3,136.7,136.8,137.6,138.1,129.8,130.8,125.3,134.2,145.3,150.1
7,Greece,GRC,186.9,194.1,200.8,198.1,194.3,189.7,179.6,171.6,182.7,177.0,173.0,178.4,177.2,173.0
8,Spain,ESP,1070.6,1071.9,1044.9,990.5,948.6,917.2,903.0,900.6,921.1,951.1,992.0,1035.1,1059.7,1103.7
9,France,FRA,1538.7,1493.6,1454.3,1457.6,1511.9,1568.5,1526.9,1550.0,1543.8,1599.0,1638.7,1688.1,1709.7,1722.1


In [20]:
low_F.to_csv('low_F.csv', index = False, na_rep = 'nan')
medium_F.to_csv('medium_F.csv', index = False, na_rep = 'nan')
high_F.to_csv('high_F.csv', index = False, na_rep = 'nan')

## Education level: percentage

We import the 18 sheets of the *pop_edu_sex_age_percentage* excel file (skipping useless rows/columns and avoiding the print of unharmful warnings) in a dictionary of datasets named *education_per_sheets*.

In [ ]:
sheet_names = [f'Sheet {i}' for i in range(1, 19)]
file_path = '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_percentage.xlsx'
rows_to_skip = list(range(0, 10)) + [11, 12, 13, 44] + list(range(49, 56))
selected_columns = ['TIME'] + [str(year) for year in range(2009, 2023)]

# Just to suppress the unharmful warnings we were getting
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = UserWarning, module = "openpyxl")
    # Dictionary with datasets: 'education_per_sheet1', ..., 'education_per_sheet36'
    education_per_sheets = {f'education_per_sheet{i}': pd.read_excel(file_path, sheet_name = sheet_name, skiprows = rows_to_skip, na_values = ':', usecols = selected_columns)
                            for i, sheet_name in enumerate(sheet_names, start = 1)}

We add a column *Abbreviation* to the 18 datasets.

In [ ]:
for i in range(1, 19):
    education_per_sheets[f'education_per_sheet{i}'].rename(columns = {'TIME': 'Country'}, inplace = True)
    education_per_sheets[f'education_per_sheet{i}']['Abbreviation'] = education_per_sheets[f'education_per_sheet{i}']['Country'].map(country_abbreviations)

## Enrolment by age

In [ ]:
enrolment = pd.read_csv("/content/drive/MyDrive/DV_project/story2/enrolment_by_age.csv")

In [ ]:
enrolment

,COUNTRY,Country,SEX,Gender,AGE,Age,INTENSITY,Intensity,EDUCATION_LEV,Education level,YEAR,Year,Value,Flag Codes,Flags
0,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2013,2013,NaN,m,Missing value; data cannot exist
1,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2014,2014,NaN,m,Missing value; data cannot exist
2,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2015,2015,NaN,m,Missing value; data cannot exist
3,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2016,2016,NaN,m,Missing value; data cannot exist
4,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2017,2017,NaN,m,Missing value; data cannot exist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524155,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2017,2017,55553.0,NaN,NaN
524156,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2018,2018,57230.0,NaN,NaN
524157,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2019,2019,48242.0,NaN,NaN
524158,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2020,2020,42941.0,NaN,NaN


# Download all CSV files

We download all the CSV files we created.

In [53]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname = os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>